In [3]:
import sys
sys.path.append('../')
sys.path.append('../src')

In [4]:
from src.qlearning import Qlearning
import matplotlib.pyplot as plt

# Set up agent

In [5]:
class Player:
    def __init__(self, get_action) -> None:
        self.get_action = get_action

# Set up training

In [6]:
Q = Qlearning()
Q.training(n_training_game=100)
Q.play()

100%|██████████| 100/100 [00:00<00:00, 151.60it/s]


# Analyze training

In [8]:
Q.plot_training_stats()
plt.show()

ValueError: v cannot be empty

# Look at a game

In [ ]:
Q.play()

In [ ]:
player = Player(get_action=lambda state: Q.get_action(state))